In [10]:
# Install dependencies
%pip install --upgrade pip 
%pip install pandas beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from bs4 import BeautifulSoup as bs
import os, json, re, hashlib

In [12]:
input_files = ["raw_html_despachosCBBA.html","raw_html_despachoSCZ.html","raw_html_reportedespachos.html"]
data_dir = '../data'

In [15]:
for input_file in input_files:
    with open(os.path.join(data_dir,input_file), 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = bs(html_content, "html.parser")
    # Find the table in the HTML
    table = soup.find("table")
    if table:
        print("Table found!")
    else:
        print("No table found in the HTML content.")

    # Extract the table headers
    headers = [header.text.strip().lower().replace(" ", "_").replace(".","") for header in table.find_all("th")]
    print("Table headers:", headers)

    # Extract the table rows
    rows = []

    for row in table.find_all("tr"):
        cells = row.find_all("td")
        if cells:
            rows.append([cell.text.strip() for cell in cells])
    print("Number of rows extracted:", len(rows))

    # Display the first few rows of the DataFrame
    obtain_time_selector = "html body div.container div.footer p"
    obtain_time = soup.select_one(obtain_time_selector)
    pattern = r"\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}"
    match = re.search(pattern, obtain_time.text)
    if match:
        print("Extracted date and time:", match.group())

    # Acumular las filas de todos los archivos y añadir la fuente
    if 'all_data' not in globals():
        all_data = []
    all_headers = headers + ['source', 'timestamp']  # Asumimos que los headers son iguales en todos los archivos y añadimos columnas para la fuente y timestamp
    rows_with_source = [row + [input_file, match.group()] for row in rows]
    all_data.extend(rows_with_source)

    # Si es el último archivo, crear el DataFrame combinado y guardarlo a CSV
    if input_file == input_files[-1]:
        import pandas as pd
        df = pd.DataFrame(all_data, columns=all_headers)
        output_file = os.path.join(data_dir, "dispatches.csv")
        df.to_csv(output_file, index=False)
        print(f"Combined data saved to {output_file}")

Table found!
Table headers: ['destino', 'localidad', 'producto', 'hora_salida', 'hora_llegada']
Number of rows extracted: 34
Extracted date and time: 15/06/2025 15:01:56
Table found!
Table headers: ['destino', 'localidad', 'producto', 'hora_salida', 'hora_llegada']
Number of rows extracted: 66
Extracted date and time: 15/06/2025 15:01:10
Table found!
Table headers: ['destino', 'localidad', 'producto', 'hora_salida', 'hora_llegada_aprox']
Number of rows extracted: 25
Extracted date and time: 15/06/2025 15:00:27
Combined data saved to ../data/dispatches.csv


In [ ]:
# obtain metadata
def obtain_metadata(file_path):
    with open(file_path, 'rb') as f:
        file_content = f.read()
    file_hash = hashlib.sha256(file_content).hexdigest()
    return {
        "file_name": os.path.basename(file_path),
        "file_size": os.path.getsize(file_path),
        "file_hash": file_hash
    }
metadata = obtain_metadata(os.path.join(data_dir, "dispatches.csv"))

# Save metadata to a JSON file
metadata_file = os.path.join(data_dir, "dispatches_metadata.json")
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=4)
print(f"Metadata saved to {metadata_file}")

# Display metadata
print("Metadata:")
print(json.dumps(metadata, indent=4))